In [1]:
import spacy
import re
import string
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS

In [2]:
nlp = spacy.load("en_core_web_md")

#Предобработка текста
def preprocess(text):
    text = text.lower().strip() #приводим к нижнему регистру и избавляеся от лишних пробелов
    text = re.sub(r'<.*?>', '', text)#убираем htlm таги
    text = re.sub(r'[^\w\s]', ' ', text) #убираем пунктицию и символы как ',", # итд
    text = re.sub(r'\s+', ' ', text)# убираем доп пробелы
    return text

#Убирает Шумовые слова как: этом, пока, одном итд
def stopword(string):
    a = [word for word in string.split() if word not in STOP_WORDS]
    return ' '.join(a)

#Приводит слово к изначальному типу
def lemmatizer(string):
    a = nlp(string)
    a = [word.lemma_ for word in a]
    return " ".join(a)

In [12]:
# открываем датасет
df = pd.read_csv("./data/Reddit_Data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37249 entries, 0 to 37248
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   clean_comment  37149 non-null  object
 1   category       37249 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 582.1+ KB


In [14]:
# удаляем лишние колонки
processed_df = df.dropna().reset_index()
processed_df.columns = ['index','text','sentiment']

In [15]:
print(processed_df['sentiment'].value_counts())   

sentiment
 1    15830
 0    13042
-1     8277
Name: count, dtype: int64


In [16]:
#создаем новый столбец с обработанным текстом
processed_df['new_text'] = processed_df.apply(lambda x: lemmatizer(stopword(preprocess(x['text']))), axis=1)

In [18]:
#создаем новый датасет
processed_df.to_csv('./data/reddit_processed.csv', encoding='utf-8')